In [1]:
%%capture
%load_ext autoreload
%autoreload 2

In [2]:
# region General Imports
import os
import uuid
import shutil
import time
import random
import datetime
import glob
import pickle
import tqdm
import copy
import optuna
import numpy as np
import pandas as pd
import scipy as sp
import matplotlib.pyplot as plt
import seaborn as sns
import mne
from rich import print as rprint
from rich.pretty import pprint as rpprint
from tqdm import tqdm
from itertools import chain
from functools import partial
# endregion General Imports

from model_optim.model_optimizer import ModelOptimizer

# Dataset
from custom_datasets.fatigue_mi import FatigueMI
from custom_datasets.norm_cho import NormCho2017
from custom_datasets.opt_game_mi import OptGameMI
from custom_datasets.opt_std_mi import OptStdMI

2024-04-11 12:47:24.395031: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:9261] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2024-04-11 12:47:24.395087: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:607] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2024-04-11 12:47:24.423098: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1515] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2024-04-11 12:47:24.502270: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2024-04-11 12:47:25.483677: W tensorflow/compiler/tf2

To use the get_shape_from_baseconcar, InputShapeSetterEEG, BraindecodeDatasetLoaderyou need to install `braindecode`.`pip install braindecode` or Please refer to `https://braindecode.org`.


/home/araz/anaconda3/lib/python3.11/site-packages/moabb/pipelines/__init__.py:26: ModuleNotFoundError: Tensorflow is not installed. You won't be able to use these MOABB pipelines if you attempt to do so.
  warn(


In [3]:
MODELS_LIST = [
    # "shallow_conv_net",
    "lstm_net",
    "deep_conv_net",
    "eeg_net",
    "lstm_cnn_net",
    "lstm_cnn_net_v2"
]
MODELS_HYPERPARAMS_DICT = {
    "shallow_conv_net": {
        "max_epochs": 5
    },
    "eeg_net": {
        "max_epochs": 10
    },
    "deep_conv_net": {
        "max_epochs": 10
    },
    "lstm_net": {
        "max_epochs": 10
    },
    "lstm_cnn_net": {
        "max_epochs": 10
    },
    "lstm_cnn_net_v2": {},
}

DATASETS_LIST = [
    FatigueMI,
    # NormCho2017,
    # OptGameMI,
    # OptStdMI,
]

In [4]:
# from model_optim.utils import data_generator

# data_generator(
#     dataset=NormCho2017(),
#     subjects=[1],
#     channel_idx=[],
#     sfreq=128,
# )

In [5]:
subject_files = glob.glob("./temp_v2/*/*/model/study_best_trial.npy")
subject_files_data = {}
for subject_file in subject_files:
    subject_files_data[subject_file] = np.load(subject_file, allow_pickle=True).item()


In [6]:
for subject_file_data_key, subject_file_data in subject_files_data.items():
    rprint(subject_file_data_key, subject_file_data.user_attrs['trial_data']['test_accuracy'])

./temp_v2/[6]/39d01251ff494106bf04f8a2cffcdd74/model/study_best_trial.npy 0.5

./temp_v2/[14]/a3304348c7094d02a024828ede942cda/model/study_best_trial.npy 0.5909091234207153

./temp_v2/[5]/d199c9c2ac924b238693f158eb88f675/model/study_best_trial.npy 0.6363636255264282

./temp_v2/[13]/1b189965ada44ff99e73fa145cd3901d/model/study_best_trial.npy 0.6363636255264282

./temp_v2/[4]/3623cb4ba1ad4a908c9098f5297a6778/model/study_best_trial.npy 0.8636363744735718

...

./temp_v2/[2]/6405d11e654b42aca9df48458c67ecde/model/study_best_trial.npy 0.6363636255264282

./temp_v2/[8]/9fd82ec44ef3496da6307b57ecf4532f/model/study_best_trial.npy 0.6363636255264282

./temp_v2/[10]/c2cc69dca74d4bfa81722cd634e6403e/model/study_best_trial.npy 0.7727273106575012

./temp_v2/[12]/96dc576945fb4f2db582d66ae1d2c8ce/model/study_best_trial.npy 0.7727273106575012

./temp_v2/[11]/e0643f9a780146a4adc15ddd4a9ff053/model/study_best_trial.npy 0.7272727489471436

In [7]:
for dataset in [FatigueMI]:
    for model in ["eeg_net"]:
        model_optimizer = ModelOptimizer(
            dataset=dataset(),
            model_name=model
        )
        for subject in [12]:
            max_epochs = MODELS_HYPERPARAMS_DICT[model]["max_epochs"]
            study = model_optimizer.search_best_model(
                subjects = [subject],
                max_iter = 25,
                max_epochs = max_epochs,
                max_stag_count = 10,
                rounds = 1,
                replace_previous_study_for_subjects = False
            )

Original sfreq: 300.0


  0%|          | 0/25 [00:00<?, ?it/s]

Adding metadata with 3 columns
Epoch 1/10


2024-04-11 12:47:30.275391: I external/local_xla/xla/stream_executor/cuda/cuda_executor.cc:887] could not open file to read NUMA node: /sys/bus/pci/devices/0000:13:00.0/numa_node
Your kernel may have been built without NUMA support.
2024-04-11 12:47:30.414748: I external/local_xla/xla/stream_executor/cuda/cuda_executor.cc:887] could not open file to read NUMA node: /sys/bus/pci/devices/0000:13:00.0/numa_node
Your kernel may have been built without NUMA support.
2024-04-11 12:47:30.414828: I external/local_xla/xla/stream_executor/cuda/cuda_executor.cc:887] could not open file to read NUMA node: /sys/bus/pci/devices/0000:13:00.0/numa_node
Your kernel may have been built without NUMA support.
2024-04-11 12:47:30.416426: I external/local_xla/xla/stream_executor/cuda/cuda_executor.cc:887] could not open file to read NUMA node: /sys/bus/pci/devices/0000:13:00.0/numa_node
Your kernel may have been built without NUMA support.
2024-04-11 12:47:30.416496: I external/local_xla/xla/stream_executor

2/2 [==============================] - ETA: 0s - loss: 4.2829 - accuracy: 0.3971
Epoch 00001: val_accuracy improved from -inf to 0.50000, storing weights.
2/2 [==============================] - 5s 410ms/step - loss: 4.2829 - accuracy: 0.3971 - val_loss: 1.3055 - val_accuracy: 0.5000 - lr: 0.0010
Epoch 2/10
1/2 [==============>...............] - ETA: 0s - loss: 1.2678 - accuracy: 0.6406
Epoch 00002: val_accuracy did not improve
2/2 [==============================] - 0s 28ms/step - loss: 1.2675 - accuracy: 0.6324 - val_loss: 1.2439 - val_accuracy: 0.4444 - lr: 0.0010
Epoch 3/10
1/2 [==============>...............] - ETA: 0s - loss: 1.1978 - accuracy: 0.5625
Epoch 00003: val_accuracy improved from 0.50000 to 0.55556, storing weights.
2/2 [==============================] - 0s 30ms/step - loss: 1.1984 - accuracy: 0.5588 - val_loss: 1.1956 - val_accuracy: 0.5556 - lr: 0.0010
Epoch 4/10
1/2 [==============>...............] - ETA: 0s - loss: 1.1316 - accuracy: 0.7344
Epoch 00004: val_accuracy 

In [8]:
model_optimizer.get_study_metrics(study).sort_values(by="scores", ascending=True)

,train_acc,test_acc,val_acc,train_val_acc_diff,train_loss,val_loss,train_val_loss_diff,test_loss,scores,channels_selected,sfreq,batch_size
0,0.794118,0.545455,0.666667,0.127451,0.868390,1.005088,0.136698,1.005517,0.111561,"[F4, C4, P4, Fp1, T3, T5, O1, O2, T4]",128.0,64
5,0.779412,0.545455,0.555556,0.223856,0.889572,0.986514,0.096942,1.105826,0.198131,"[P3, C3, C4, Cz, Pz, Fp1, O1, O2, F7, F8, T6, T4]",128.0,64
1,0.808824,0.818182,0.666667,0.142157,1.008998,1.079126,0.070128,1.078813,0.861611,"[Fz, C4, P4, Cz, Fp2, T3, O2, F7, A2, T4]",128.0,96
6,0.705882,0.500000,0.555556,0.150327,0.849370,0.934690,0.085320,1.202529,0.947931,"[F3, Fz, F4, Fp2, T5, F8, A2, T4]",128.0,64
9,0.647059,0.500000,0.500000,0.147059,0.890274,0.890007,0.000267,0.967755,1.000750,"[P3, F3, Fz, C4, P4, Cz, Pz, Fp2, T5, O1, O2, ...",128.0,32
2,0.720588,0.500000,0.444444,0.276144,1.104852,1.121118,0.016266,1.960047,1.059242,"[P3, C3, F3, Fz, F4, P4, Cz, Pz, T3, O2, F7, A2]",256.0,128
7,0.779412,0.500000,0.444444,0.334967,1.495029,1.580115,0.085086,2.203085,1.059242,"[P3, F3, F4, C4, P4, Cz, Pz, Fp2, T5, O2, T6, T4]",NaN,160
8,0.588235,0.500000,0.611111,0.022876,1.145460,1.109649,0.035811,1.311961,1.151785,"[C3, Fz, P4, Pz, Fp2, T3, T5, O1, F7, A2, T4]",NaN,64
3,0.779412,0.500000,0.555556,0.223856,1.106127,1.177586,0.071458,1.895464,1.197831,"[C3, F3, C4, Cz, Pz, O2]",NaN,160
4,0.705882,0.500000,0.555556,0.150327,0.915680,0.948252,0.032573,1.063092,1.197931,"[F3, Fz, Pz, Fp1, Fp2, O1, F7, T6]",256.0,64


In [9]:
rpprint({ k: v for k, v in study.best_trial.params.items() if not k.startswith("channels") })
rprint("test_accuracy =", study.best_trial.user_attrs["trial_data"]["test_accuracy"])
rprint("val_accuracy =", np.max(study.best_trial.user_attrs["trial_data"]["val_accuracy"]))
rprint("channels_selected =", study.best_trial.user_attrs["trial_data"]["channels_selected"])

{
│   'sfreq': 128,
│   'batch_size': 64,
│   'conv2d_1_units': 40,
│   'conv2d_1_kernl_length': 16,
│   'pool_1_size': 8,
│   'conv2d_depth_multiplier': 8,
│   'conv2d_2_units': 64,
│   'conv2d_2_kernl_length': 80,
│   'pool_2_size': 8,
│   'l2_reg_1': 0.32825455382862395,
│   'l2_reg_2': 0.5696184885654687,
│   'l2_reg_3': 0.485802254049597,
│   'l2_reg_4': 0.8027068971954714,
│   'dropout_rate': 0.4
}

test_accuracy = 0.5454545617103577

val_accuracy = 0.6666666865348816

channels_selected = ['F4' 'C4' 'P4' 'Fp1' 'T3' 'T5' 'O1' 'O2' 'T4']